In [90]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import nltk.data
import collections

In [91]:
filenames = ["data/reviews_Home_and_Kitchen_5.json"]
filename = "data/reviews_Home_and_Kitchen_5.json"
file_name = "rawdata_100.pkl"
filename_adversarial_wordnet = "data_adversial_wordnet.pkl"
filename_adversarial_glove = "data_adversial_glove.pkl"
max_features = 20000
maxlen = 100
batch_size = 10
num_class = 2

set_adversial = []
set_unchanged = []

In [92]:
data = pd.read_pickle(file_name)

In [93]:
data.head(10)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
44007,B000079R7G,"[9, 14]",0,i bought this based on other's positive review...,"05 20, 2006",AZU2JDR2GVICQ,"Zyde ""zyde""",Horrible and Expensive,1148083200
126671,B000B649X2,"[0, 0]",0,"i had seen these in stores, and when amazon of...","03 19, 2006",A1O3LAET0L3BHU,Jo Gusto,Fantastic concept... miserable battery perform...,1142726400
412572,B004NGVIIK,"[0, 0]",1,i ordered these right along with the pressure ...,"10 25, 2013",A6RGPU0H5VFTZ,melissa baldwin,good deal,1382659200
131315,B000BR391E,"[0, 0]",0,this was a great deal and worked super until s...,"05 14, 2011",ATJDQ137MA426,L. Mitchell,Timer Stopped Working,1305331200
133376,B000C239I6,"[0, 0]",1,i received this set last christmas (2012) and ...,"08 21, 2013",A24X6MADWDYX8T,Lisa C.,Best set of cookware imaginable,1377043200
484182,B0081JZE1W,"[2, 2]",0,the filter pictured is that of an oem dyson pr...,"03 17, 2014",A1L1XP2KQ99BGJ,Bassinvol,Not an OEM product as pictured,1395014400
105172,B0006HUYGM,"[10, 10]",1,i agree with the five star commenters -- this ...,"09 29, 2005",A2O6AAW9WZ1XJB,"Virginia music lover ""cossack""",Excellent product for a low price,1127952000
125552,B000ASDCWK,"[3, 3]",1,the only complaint i may have on this item is ...,"10 20, 2006",AB0NEAUW9KJZR,dbstyle,Great,1161302400
397462,B004918OPO,"[0, 0]",1,this pitcher has saved me a bundle on bottled ...,"07 22, 2013",A1G0RMYT6CHP66,"L. Nelson ""Big Sis""",love it,1374451200
34781,B0000632QE,"[665, 683]",1,i purchased the v-slicer over ten years ago an...,"07 21, 2002",A3D0RU1URR2IOF,Daniel F. Moore,Hasn't left the counter in ten years,1027209600


### replace nouns with antonyms and adj with synonyms

In [94]:
POS_TO_WORDNET = {
    'NN': wn.NOUN,
    'JJ': wn.ADJ,
    'JJR': wn.ADJ,
    'JJS': wn.ADJ,
}

In [95]:
def alter_wordnet_antonyms(token):
    if token[1] not in POS_TO_WORDNET: return None
    w = token[0].lower()
    wn_pos = POS_TO_WORDNET[token[1]]
    synsets = wn.synsets(w, wn_pos)
    if not synsets: return None
    synset = synsets[0]
    synonyms = []
    antonyms = []

    for syn in wn.synsets(w, wn_pos):
        for lem in syn.lemmas():
            if (lem.name() != w):
                synonyms.append(lem.name())
#     for lem in synset.lemmas():
#         if lem.antonyms():
# #                 print("nouns: ", i)
#             for a in lem.antonyms():
#                 new_word = a.name()
#                 if '_' in a.name(): continue
#                 antonyms.append(new_word)
#         else:
#             print("adj: ", i)
#         synonyms.append(lem.name())
#     if token[1] == 'NN':
#         return antonyms
#     else:
    return synonyms

In [96]:
texts = data['reviewText']
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [97]:
numSample = 3
for i, row in data.iterrows():
    text = texts[i]
    tokenized = tokenizer.tokenize(text)
    words = word_tokenize(text)
    tokens = nltk.pos_tag(words)
    output = ""
    index = 0
    isChanged = False
    for token in tokens:
        if index != 0:
            output = output + ' '
        synset = alter_wordnet_antonyms(token)
        if synset is None or len(synset) == 0:
            output = output + token[0]
        else:
            isChanged = True
            if (numSample > 0):
                print(i, ": ", token[0], " to ", synset[0])
            output = output + synset[0]
        index = index + 1
    if isChanged:
        if (numSample > 0):
            print(output)
            print("")
            numSample = numSample - 1
        data.at[i,'reviewText'] = output
        set_adversial.append(i)
    else:
        set_unchanged.append(i)

44007 :  i  to  iodine
44007 :  other  to  early
44007 :  positive  to  convinced
44007 :  share  to  portion
44007 :  same  to  like
44007 :  positive  to  convinced
44007 :  horrible  to  atrocious
44007 :  suction  to  sucking
44007 :  horrible  to  atrocious
44007 :  hair  to  hair's-breadth
44007 :  i  to  iodine
44007 :  old  to  older
44007 :  boss  to  foreman
44007 :  better  to  best
44007 :  cheaper  to  cheap
44007 :  hair  to  hair's-breadth
44007 :  couch  to  sofa
44007 :  first  to  1st
44007 :  i  to  iodine
44007 :  roller  to  roll
44007 :  rolling  to  rolled
44007 :  lower  to  low
44007 :  surface  to  Earth's_surface
44007 :  small  to  little
44007 :  whatnot  to  bric-a-brac
44007 :  suction  to  sucking
44007 :  carpet  to  rug
44007 :  carpet  to  rug
44007 :  vacuum  to  vacuity
44007 :  little  to  small
44007 :  thing  to  matter
44007 :  main  to  chief
44007 :  roller  to  roll
44007 :  loud  to  brassy
44007 :  warning  to  admonition
44007 :  sound  to

In [101]:
len(set_adversial)

102664

In [102]:
len(data)

102838

In [104]:
data = data.loc[set_adversial,:]

In [105]:
data.head(10)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
44007,B000079R7G,"[9, 14]",0,iodine bought this based on early 's convinced...,"05 20, 2006",AZU2JDR2GVICQ,"Zyde ""zyde""",Horrible and Expensive,1148083200
126671,B000B649X2,"[0, 0]",0,"iodine had seen these in stores , and when vir...","03 19, 2006",A1O3LAET0L3BHU,Jo Gusto,Fantastic concept... miserable battery perform...,1142726400
412572,B004NGVIIK,"[0, 0]",1,iodine ordered these correct along with the pr...,"10 25, 2013",A6RGPU0H5VFTZ,melissa baldwin,good deal,1382659200
131315,B000BR391E,"[0, 0]",0,this was a outstanding trade and worked ace un...,"05 14, 2011",ATJDQ137MA426,L. Mitchell,Timer Stopped Working,1305331200
133376,B000C239I6,"[0, 0]",1,i received this set concluding Christmas ( 201...,"08 21, 2013",A24X6MADWDYX8T,Lisa C.,Best set of cookware imaginable,1377043200
484182,B0081JZE1W,"[2, 2]",0,the filter pictured is that of an oem dyson pr...,"03 17, 2014",A1L1XP2KQ99BGJ,Bassinvol,Not an OEM product as pictured,1395014400
105172,B0006HUYGM,"[10, 10]",1,i agree with the five ace commenters -- this i...,"09 29, 2005",A2O6AAW9WZ1XJB,"Virginia music lover ""cossack""",Excellent product for a low price,1127952000
125552,B000ASDCWK,"[3, 3]",1,the lone ailment iodine may have on this point...,"10 20, 2006",AB0NEAUW9KJZR,dbstyle,Great,1161302400
397462,B004918OPO,"[0, 0]",1,this hurler has saved me a package on bottled ...,"07 22, 2013",A1G0RMYT6CHP66,"L. Nelson ""Big Sis""",love it,1374451200
34781,B0000632QE,"[665, 683]",1,iodine purchased the v-slicer over ten years a...,"07 21, 2002",A3D0RU1URR2IOF,Daniel F. Moore,Hasn't left the counter in ten years,1027209600


In [106]:
len(data.loc[set_adversial,:])

102664

In [107]:
data.to_pickle(filename_adversarial_wordnet)

In [78]:
### replace named entity and numbers]

In [ ]:
data = pd.read_pickle(file_name)

In [76]:
data.head(10)

,overall,reviewText
489257,1,I use this set for both general food storage i...
318868,0,Not a huge fan . I like to have a side for dam...
527578,1,I have always wanted to have a French press at...
102291,0,I purchased this to replace a breadmaker that ...
137178,0,I bought this pan because my 12 '' pan needed ...
233040,0,"Just sayin ' . I mean , honestly . Do we reall..."
413128,1,"This rack fits the sink perfectly , and is n't..."
149988,0,I bought one of the 20X72 inch mats for my hus...
18029,1,I have four different sizes of these scoops an...
129849,1,These replacement rubbers are perfect for use ...


In [77]:
data = pd.read_pickle(adversarial_file_name)

In [78]:
data.head(10)

,overall,reviewText
489257,1,I use this set for both general food storage i...
318868,0,Not a huge fan . I like to have a side for dam...
527578,1,I have always wanted to have a French press at...
102291,0,I purchased this to replace a breadmaker that ...
137178,0,I bought this pan because my 12 '' pan needed ...
233040,0,"Just sayin ' . I mean , honestly . Do we reall..."
413128,1,"This rack fits the sink perfectly , and is n't..."
149988,0,I bought one of the 20X72 inch mats for my hus...
18029,1,I have four different sizes of these scoops an...
129849,1,These replacement rubbers are perfect for use ...


In [72]:
wn.synsets('dog') # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [73]:
wn.synsets('dog', pos=wn.VERB)

[Synset('chase.v.01')]

In [74]:
wn.synset('dog.n.01')

Synset('dog.n.01')

In [75]:
print(wn.synset('dog.n.01').definition())

a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds


In [76]:
len(wn.synset('dog.n.01').examples())

1

In [77]:
print(wn.synset('dog.n.01').examples()[0])

the dog barked all night


In [78]:
wn.synset('dog.n.01').lemmas()

[Lemma('dog.n.01.dog'),
 Lemma('dog.n.01.domestic_dog'),
 Lemma('dog.n.01.Canis_familiaris')]

In [79]:
[str(lemma.name()) for lemma in wn.synset('dog.n.01').lemmas()]

['dog', 'domestic_dog', 'Canis_familiaris']

In [80]:
wn.lemma('dog.n.01.dog').synset()

Synset('dog.n.01')

In [85]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vwslz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [ ]:
data.to_pickle(adversarial_file_name)